# Annotation of Erdman genome
This notebook aims to create a high quality annotation of the Erdman genome. The approach is the following:
1. Extract annotation features from the RATT annotation made based on H37Rv.
2. Extract annotation features made _de novo_ with Bakta.
3. From the Bakta annotations, keep only those non-overlapping with RATT annotation
4. Build genbank record
5. Manually fix CDS transferred with invalid ORFs, add translations and assign final Erdman locus tag.
6. Build feature table
7. Run quality control

In [1]:
from Bio import SeqIO
from io import StringIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pandas as pd
import bioframe as bf
from Bio.Seq import Seq

## 1. Extract features from RATT annotation
Made based on H37Rv

In [2]:
def fix_embl_id_line(filename):
    '''
    ratt makes strange ID line in the embl file,
    that biopython doesn't like. This is a quick fix to read it.
    '''
    with open(filename, 'r') as file:
        lines = file.readlines()

    # new_first_line = 'ID                   Erdmann; ; ; ; ; ; 0 BP.'
    new_first_line = 'ID   Erdmann; ; circular; genomic DNA; ; ; 0 BP.'
    lines[0] = new_first_line + '\n'
    modified_embl = ''.join(lines)
    
    return StringIO(modified_embl)

In [3]:
ratt_file = '../../Results/RATT/RATT_AssemblyRepetitive/Erdmann.Assembly.Repetitive.contig_1.final.embl'
bakta_file = '../../Results/Bakta_annotation/result.gbff'

In [4]:
# write ratt embl output to gb
ratt_gb = '../../Results/RATT/RATT_AssemblyRepetitive/Erdmann.Assembly.Repetitive.contig_1.final.gb'
SeqIO.write(list(SeqIO.parse(fix_embl_id_line(ratt_file), 'embl')), ratt_gb, 'genbank')

1

In [5]:
# make a dictionary with the genes transferred by ratt
# rv_genes_in_erdman[locus_tag] = location

rv_genes_in_erdman = {}

for record in SeqIO.parse(fix_embl_id_line(ratt_file), 'embl'):
    for feature in record.features:
        # get only gene features
        if feature.type == 'gene':
            
            # get rv locus tag
            rv_locus_tag = feature.qualifiers['locus_tag'][0]

            # add location and strand
            rv_genes_in_erdman[rv_locus_tag] = {}
            rv_genes_in_erdman[rv_locus_tag]['start'] = feature.location.start
            rv_genes_in_erdman[rv_locus_tag]['end'] = feature.location.end
            rv_genes_in_erdman[rv_locus_tag]['strand'] = feature.location.strand   


ratt_features = []

for record in SeqIO.parse(fix_embl_id_line(ratt_file), 'embl'):
    for feature in record.features:
        # ignore these features:
        if feature.type not in ['source', 'repeat_region', 'gene', 'mobile_element']:
            
            # make a new feature with selected information

            # get location
            new_feature_start = feature.location.start
            new_feature_end = feature.location.end
            new_feature_strand = feature.location.strand

            if new_feature_strand == 1:
                new_feature_strand_symbol = '+'
            elif new_feature_strand == -1:
                new_feature_strand_symbol = '-'

            # make dictionary to store qualifiers of feature
            new_feature_qualifiers = feature.qualifiers

            # add if there's an Rv locus tag
            if 'locus_tag' in feature.qualifiers.keys():
                rv_id = feature.qualifiers['locus_tag'][0]
            else:
                rv_id = None
 
            # fix the CDS start
            
            # if there's a gene annotation with the same rv_id
            if rv_id in rv_genes_in_erdman.keys():
                # if the strand is +, update the "start" with what the rv_genes_in_erdman dictionary says
                if new_feature_strand == 1:
                    new_feature_start = rv_genes_in_erdman[rv_id]['start']
                elif new_feature_strand == -1:
                    # if the strand is -, update the "end"
                    new_feature_end = rv_genes_in_erdman[rv_id]['end']
                
            # make new feature
            new_feature = SeqFeature(FeatureLocation(new_feature_start, 
                                                     new_feature_end, 
                                                     strand=new_feature_strand),
                                     type=feature.type,
                                     qualifiers=new_feature_qualifiers)


            feature_line = ['Erdman', feature.type, new_feature_start+1, 
                            new_feature_end, new_feature_strand_symbol, 
                            rv_id, 'ratt', new_feature]
            ratt_features.append(feature_line)

df_ratt = pd.DataFrame(ratt_features, columns=['chrom', 'type', 'start', 'end', 
                                               'strand', 'rv_equivalent', 'source', 'seqrecord'])

In [6]:
df_ratt.head()

,chrom,type,start,end,strand,rv_equivalent,source,seqrecord
0,Erdman,CDS,1,1524,+,Rv0001,ratt,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,Erdman,CDS,2052,3260,+,Rv0002,ratt,"(2051, 2052, 2053, 2054, 2055, 2056, 2057, 205..."
2,Erdman,CDS,3280,4437,+,Rv0003,ratt,"(3279, 3280, 3281, 3282, 3283, 3284, 3285, 328..."
3,Erdman,CDS,4434,4997,+,Rv0004,ratt,"(4433, 4434, 4435, 4436, 4437, 4438, 4439, 444..."
4,Erdman,CDS,5240,7267,+,Rv0005,ratt,"(5239, 5240, 5241, 5242, 5243, 5244, 5245, 524..."


In [7]:
df_ratt['seqrecord'].apply(lambda x: x.type).value_counts()

seqrecord
CDS             3877
tRNA              45
misc_feature      33
ncRNA             20
rRNA               3
misc_RNA           2
Name: count, dtype: int64

## 2. Extract annotation features made _de novo_ with Bakta.

In [8]:
bakta_features = []

for record in SeqIO.parse(bakta_file, 'genbank'):
    for feature in record.features:
        # ignore these features:
        if feature.type not in ['repeat_region', 'gene', 'mobile_element', 'source']:
            
            # make a new feature with selected information

            # get location
            new_feature_start = feature.location.start
            new_feature_end = feature.location.end
            new_feature_strand = feature.location.strand

            if new_feature_strand == 1:
                new_feature_strand_symbol = '+'
            elif new_feature_strand == -1:
                new_feature_strand_symbol = '-'

            # make dictionary to store qualifiers of feature
            new_feature_qualifiers = feature.qualifiers
                
            # make new feature
            new_feature = SeqFeature(FeatureLocation(new_feature_start, 
                                                     new_feature_end, 
                                                     strand=new_feature_strand),
                                     type=feature.type,
                                     qualifiers=new_feature_qualifiers)


            feature_line = ['Erdman', feature.type, new_feature_start+1, new_feature_end, new_feature_strand_symbol, None, 'bakta', new_feature]
            bakta_features.append(feature_line)

df_bakta = pd.DataFrame(bakta_features, 
                       columns=['chrom', 'type', 'start', 'end', 'strand', 
                                'rv_equivalent', 'source', 'seqrecord'])

In [9]:
!ls -1 ../../Results/Bakta_annotation/

result.embl
result.faa
result.ffn
result.fna
result.gbff
result.gff3
result.json
result.tsv


In [10]:
df_bakta.head()

,chrom,type,start,end,strand,rv_equivalent,source,seqrecord
0,Erdman,CDS,1,1524,+,None,bakta,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,Erdman,rep_origin,1525,2051,+,None,bakta,"(1524, 1525, 1526, 1527, 1528, 1529, 1530, 153..."
2,Erdman,CDS,2052,3260,+,None,bakta,"(2051, 2052, 2053, 2054, 2055, 2056, 2057, 205..."
3,Erdman,CDS,3280,4437,+,None,bakta,"(3279, 3280, 3281, 3282, 3283, 3284, 3285, 328..."
4,Erdman,CDS,4482,4997,+,None,bakta,"(4481, 4482, 4483, 4484, 4485, 4486, 4487, 448..."


In [11]:
df_bakta['seqrecord'].apply(lambda x: x.type).value_counts()

seqrecord
CDS           4070
tRNA            46
ncRNA           23
regulatory      13
rRNA             3
rep_origin       1
tmRNA            1
Name: count, dtype: int64

## 3. From the Bakta annotations, keep only those non-overlapping with RATT annotation

In [12]:
overlaps = bf.count_overlaps(df_bakta, df_ratt)
to_transfer_from_bakta = overlaps[overlaps['count'] == 0]

df_all_features = pd.concat([df_ratt, to_transfer_from_bakta]).sort_values('start')

In [13]:
df_all_features.head()

,chrom,type,start,end,strand,rv_equivalent,source,seqrecord,count
0,Erdman,CDS,1,1524,+,Rv0001,ratt,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",NaN
1,Erdman,rep_origin,1525,2051,+,None,bakta,"(1524, 1525, 1526, 1527, 1528, 1529, 1530, 153...",0.0
1,Erdman,CDS,2052,3260,+,Rv0002,ratt,"(2051, 2052, 2053, 2054, 2055, 2056, 2057, 205...",NaN
2,Erdman,CDS,3280,4437,+,Rv0003,ratt,"(3279, 3280, 3281, 3282, 3283, 3284, 3285, 328...",NaN
3,Erdman,CDS,4434,4997,+,Rv0004,ratt,"(4433, 4434, 4435, 4436, 4437, 4438, 4439, 444...",NaN


In [14]:
df_all_features['type'].value_counts()

type
CDS             4025
tRNA              46
misc_feature      33
ncRNA             22
regulatory         6
rRNA               3
misc_RNA           2
rep_origin         1
Name: count, dtype: int64

## 4. Build genbank record
This describes the heuristics of what to transfer from the ratt and Bakta annotations

In [15]:
i = 1
new_features = []
gene_column = []
erd_locus_tag_column = []

for column, row in df_all_features.iterrows():
    seqrecord = row['seqrecord']
    qualifiers = {}
    
    if row['type']  == 'CDS':
        # locus_tag
        erdman_locus_tag = f'tmp_Erdman_{str(i).zfill(4)}'
        qualifiers['locus_tag'] = erdman_locus_tag
        i += 1

        # gene
        if 'gene' in seqrecord.qualifiers:
            qualifiers['gene'] = seqrecord.qualifiers['gene']
        elif row['source'] == 'ratt':
            if 'locus_tag' in seqrecord.qualifiers:
                rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                qualifiers['gene'] = [f'E{rv_locus_tag}']
        else:
            qualifiers['gene'] = [erdman_locus_tag]

        # product
        if 'product' in seqrecord.qualifiers:
            qualifiers['product'] = seqrecord.qualifiers['product']

        # codon_start
        if 'codon_start' in seqrecord.qualifiers:
            qualifiers['codon_start'] = seqrecord.qualifiers['codon_start']
            
        # transl_table
        if 'transl_table' in seqrecord.qualifiers:
            qualifiers['transl_table'] = seqrecord.qualifiers['transl_table']
            
        # db_xref
        if row['source'] == 'bakta':
            if 'db_xref' in seqrecord.qualifiers:
                qualifiers['db_xref'] = seqrecord.qualifiers['db_xref']

        # inference
        if row['source'] == 'ratt':
            qualifiers['inference'] = 'alignment:ratt:1.0.3:RefSeq:NC_000962.3'
        elif row['source'] == 'bakta':
            if 'inference' in seqrecord.qualifiers:
                qualifiers['inference'] = seqrecord.qualifiers['inference']

        # note
        if row['source'] == 'ratt':
            if 'locus_tag' in seqrecord.qualifiers:
                rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                qualifiers['note'] = f'Simliar to {rv_locus_tag} of M. tuberculosis H37Rv'
        elif row['source'] == 'bakta':
            qualifiers['note'] = 'De novo annotation with bakta'

    elif row['type'] in ['tRNA', 'ncRNA', 'rRNA', 'misc_RNA']:
        # locus_tag
        erdman_locus_tag = f'tmp_Erdman_{str(i).zfill(4)}'
        qualifiers['locus_tag'] = erdman_locus_tag
        i += 1
        
        # gene
        if 'gene' in seqrecord.qualifiers:
            qualifiers['gene'] = seqrecord.qualifiers['gene']
        elif row['source'] == 'ratt':
            if 'locus_tag' in seqrecord.qualifiers:
                rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                qualifiers['gene'] = [f'E{rv_locus_tag}']
        else:
            qualifiers['gene'] = [erdman_locus_tag]

        # product
        if 'product' in seqrecord.qualifiers:
            qualifiers['product'] = seqrecord.qualifiers['product']

        # db_xref
        if row['source'] == 'bakta':
            if 'db_xref' in seqrecord.qualifiers:
                qualifiers['db_xref'] = seqrecord.qualifiers['db_xref']

        # inference
        if row['source'] == 'ratt':
            qualifiers['inference'] = 'alignment:ratt:1.0.3:RefSeq:NC_000962.3'
        elif row['source'] == 'bakta':
            if 'inference' in seqrecord.qualifiers:
                qualifiers['inference'] = seqrecord.qualifiers['inference']

        # note
        if row['source'] == 'ratt':
            if 'locus_tag' in seqrecord.qualifiers:
                rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                qualifiers['note'] = f'Simliar to {rv_locus_tag} of M. tuberculosis H37Rv'
        elif row['source'] == 'bakta':
            qualifiers['note'] = 'De novo annotation with bakta'

        # special transfer only for ncRNA
        if row['type'] == 'ncRNA':
            qualifiers['ncRNA_class'] = seqrecord.qualifiers['ncRNA_class']

    elif row['type'] == 'misc_feature':
        # some are in reality pseudogenes
        if 'pseudogene' in seqrecord.qualifiers:
            seqrecord.type = 'CDS'
            
            # locus_tag
            erdman_locus_tag = f'tmp_Erdman_{str(i).zfill(4)}'
            qualifiers['locus_tag'] = erdman_locus_tag
            i += 1
            
            # gene
            if 'gene' in seqrecord.qualifiers:
                qualifiers['gene'] = seqrecord.qualifiers['gene']
            elif row['source'] == 'ratt':
                if 'locus_tag' in seqrecord.qualifiers:
                    rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                    qualifiers['gene'] = [f'E{rv_locus_tag}']
            else:
                qualifiers['gene'] = [erdman_locus_tag]

            # product
            if 'note' in seqrecord.qualifiers:
                qualifiers['product'] =  seqrecord.qualifiers['note'][0]

            # note
            if row['source'] == 'ratt':
                if 'locus_tag' in seqrecord.qualifiers:
                    rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                    qualifiers['note'] = f'Simliar to {rv_locus_tag} of M. tuberculosis H37Rv'
            elif row['source'] == 'bakta':
                qualifiers['note'] = 'De novo annotation with bakta'

            qualifiers['pseudo'] = None
            # qualifiers['pseudogene'] = 'unknown'
        else:
            seqrecord = None
        
    elif row['type'] == 'regulatory':
        # regulatory_class
        if 'regulatory_class' in seqrecord.qualifiers:
            if row['source'] == 'bakta':
                qualifiers['regulatory_class'] = seqrecord.qualifiers['regulatory_class']

        # db_xref
        if row['source'] == 'bakta':
            if 'db_xref' in seqrecord.qualifiers:
                qualifiers['db_xref'] = seqrecord.qualifiers['db_xref']

        # inference
        if row['source'] == 'ratt':
            qualifiers['inference'] = 'alignment:ratt:1.0.3:RefSeq:NC_000962.3'
        elif row['source'] == 'bakta':
            if 'inference' in seqrecord.qualifiers:
                qualifiers['inference'] = seqrecord.qualifiers['inference']

        # function
        if row['source'] == 'bakta':
            if 'note' in seqrecord.qualifiers:
                qualifiers['function'] = seqrecord.qualifiers['note']

        # note
        if row['source'] == 'ratt':
            if 'locus_tag' in seqrecord.qualifiers:
                rv_locus_tag = seqrecord.qualifiers['locus_tag'][0]
                qualifiers['note'] = f'Simliar to {rv_locus_tag} of M. tuberculosis H37Rv'
        elif row['source'] == 'bakta':
            qualifiers['note'] = 'De novo annotation with bakta'

    elif row['type'] == 'rep_origin':
        # function
        if row['source'] == 'bakta':
            if 'note' in seqrecord.qualifiers:
                qualifiers['function'] = seqrecord.qualifiers['note']

        # note
        if row['source'] == 'bakta':
            qualifiers['note'] = 'De novo annotation with bakta'

    # make new feature
    if seqrecord:
        new_feature = SeqFeature(seqrecord.location,
                                 type=seqrecord.type,
                                 qualifiers=qualifiers)
    
        new_features.append(new_feature)




##### Manually adding barcodng plasmid features #####

# Here I will manually add the barcoding plasmid features which range from 2764911 to 2770126

# 1) general annotation of barcoding plasmid region
pjeb_barcoding_vector_Feat = SeqFeature(
                        location=FeatureLocation(2764911, 2770126, strand=+1),  # Specify the start and end positions
                        type="misc_feature",  # Specify the feature type
                        qualifiers={"label" : "pjeb_barcoding_vector",
                                    "note": "Inserted barcoding plasmid region"}  # Name the feature
)

new_features.append(pjeb_barcoding_vector_Feat)

# 2) attP_core_sequence

attP_core_sequence_Feat = SeqFeature(location=FeatureLocation(2764912, 2764955, strand=+1),  # Specify the start and end positions
                                type="misc_feature",  # Specify the feature type
                                qualifiers={"label" : "attP_core_sequence",
                                            "note": "barcoding plasmid feature - attP_core_sequence"}  # Name the feature
                            )

new_features.append(attP_core_sequence_Feat)


# 3) qTag_v27
qTag_v27_Feat = SeqFeature(location=FeatureLocation(2765549, 2765720, strand=+1),  # Specify the start and end positions
                                type="misc_feature",  # Specify the feature type
                                qualifiers={"label" : "qTag_v27",
                                            "note": "barcoding plasmid feature - qTag_v27"}  # Name the feature
                            )

new_features.append(qTag_v27_Feat)

# 4) OriE
OriE_Feat = SeqFeature(location=FeatureLocation(2765750, 2766543, strand=+1),  # Specify the start and end positions
                                type="misc_feature",  # Specify the feature type
                                qualifiers={"label" : "OriE",
                                            "note": "barcoding plasmid feature - OriE"}  # Name the feature
                            )

new_features.append(OriE_Feat)


# 5) barcode_random_7mer

barcode_random_7mer_Feat = SeqFeature(location=FeatureLocation(2765720, 2765727, strand=+1),  # Specify the start and end positions
                                type="misc_feature",  # Specify the feature type
                                qualifiers={"label" : "barcode_random_7mer",
                                            "note": "barcoding plasmid feature - barcode_random_7mer"}  # Name the feature
                            )

new_features.append(barcode_random_7mer_Feat)

#################################



new_record = list(SeqIO.parse(fix_embl_id_line(ratt_file), 'embl'))[0]
new_record.id = 'Erdman'
new_record.name = 'Erdman'

source_feature = new_record.features[0]

source_feature.qualifiers = {'strain': ['Erdman'],
                                                       'mol_type': ['genomic DNA'],
                                                       'db_xref': ['taxon:652616'],
                                                       'organism': ['Mycobacterium tuberculosis strain Erdman']}
new_features.insert(0, source_feature)

new_record.features = new_features

## 5. Manually fix CDS transferred with invalid ORF.
The file 'manual_fix.tsv' was made by manually inspecting cases of invalid orfs and by fetching the right coordinates.

In [16]:
df_fix = pd.read_csv('manual_fix.tsv',
                     sep='\t')


df_fix[['start', 'end']] = df_fix[['start', 'end']].fillna(0).astype(int)

fix_dict = {}
for column, row in df_fix.iterrows():
    gene = row['gene']
    fix_dict[gene] = {
        'position': (row['start'], row['end']),
        'action': row['action'].split(';'),
        'note': row['note'] if row['note'] != 'NaN' else None
    }


clean_features = []

# clean features according to what's specified in 'action'
for feature in new_record.features:
    if 'gene' in feature.qualifiers:
        gene = feature.qualifiers['gene'][0]
        if gene in fix_dict.keys():
            for task in fix_dict[gene]['action']:
                if task == 'update_position':
                    feature.location = FeatureLocation(fix_dict[gene]['position'][0]-1,
                                                       fix_dict[gene]['position'][1],
                                                       strand=feature.location.strand)
                if task == 'change_strand':
                    if feature.location.strand:
                        feature.location.strand = -1
                    else:
                        feature.location.strand = 1
                elif task == 'add_note':
                    feature.qualifiers['note'] += fix_dict[gene]['note']
                elif task == 'pseudo':
                    feature.qualifiers['pseudo'] = ''
                elif task == 'delete':
                    feature = None

    # get translation   
    if feature:
        if feature.type == 'CDS':
            if 'pseudo' not in feature.qualifiers:
                if feature.location.strand > 0:
                    subseq = new_record.seq[feature.location.start:feature.location.end]
                    aa = subseq.translate()
                else:
                    subseq = new_record.seq[feature.location.start:feature.location.end]
                    aa = subseq.reverse_complement().translate()
                feature.qualifiers['translation'] = Seq(aa)
        
        # clean_features.append(feature)



    #### Add a label for KanR gene and L5 Integrase

        if 'gene' in feature.qualifiers:
            gene_name = feature.qualifiers["gene"][0]
            if gene_name == "aph(3')-Ia":
                feature.qualifiers["label"] = ["aph_KanR"]
                feature.qualifiers["note"] = ['Barcoding plasmid: aph_KanR, kanamycin resistance gene']
    
            if gene_name == "tmp_Erdman_2579":
                feature.qualifiers["label"] = ["L5 integrase"]
                feature.qualifiers['gene'][0] = 'L5-gp33'
                feature.qualifiers["product"][0] = "site specific integrase"
                feature.qualifiers["note"] = ["Barcoding plasmid: site specific integrase (L5 gp33) (UniProt: P22884)"]
    
            if gene_name == "tmp_Erdman_2578":
                feature.qualifiers["label"] = ["L5 transcriptional repressor"]
                feature.qualifiers['gene'][0] = 'L5-gp71'
                feature.qualifiers["product"][0] = "L5 transcriptional repressor"
                feature.qualifiers["note"] = ["Barcoding plasmid: L5 transcriptional repressor (L5 gp71) (UniProt: Q05286)"]

        clean_features.append(feature)




# RESORT THE clean_features LIST

clean_features = sorted(clean_features, key=lambda feature: feature.location.start)

# add updated locus_tag
i = 1

for feature in clean_features:
    if 'locus_tag' in feature.qualifiers:
        erdman_locus_tag = f'Erdman_{str(i).zfill(4)}'
        feature.qualifiers['locus_tag'] = erdman_locus_tag
        i += 1

In [17]:
len(clean_features)

4112

## 6. Build feature table and rename `gene` to `locus_tag`

In [18]:
annotations = []

for feature in clean_features:
    if 'locus_tag' in feature.qualifiers:
        locus_tag = feature.qualifiers['locus_tag']
    else:
        locus_tag = None
    if 'gene' in feature.qualifiers:
        gene = feature.qualifiers['gene'][0]
        if 'tmp_Erdman' in gene:
            gene = locus_tag
            feature.qualifiers['gene'][0] = locus_tag
    else:
        gene = None
    record_type = feature.type
    start = feature.location.start
    end = feature.location.end
    if feature.location.strand > 0:
        strand = '+'
    else:
        strand = '-'
    
    if 'note' in feature.qualifiers:
        note = feature.qualifiers['note']
        if 'bakta' in note:
            source = 'bakta'
            rv_equivalent = None
            
        elif 'H37Rv' in note:
            source = 'ratt'
            rv_equivalent = note.split('Simliar to ')[1].split(' of M. tuberculosis H37Rv')[0]
            
    else:
        rv_equivalent = None
        source = None
        note = ""
    line = [locus_tag, gene, record_type, rv_equivalent, start, end, strand, source, note]
    annotations.append(line)

df_annotations = pd.DataFrame(annotations, columns = ['locus_tag', 'gene', 'type', 'rv_equivalent', 'start', 'end', 'strand', 'source', 'note'])

In [19]:
df_annotations.to_csv('Erdman.curated.tsv', sep='\t', index=None)

In [20]:
# save
new_record.features = clean_features
SeqIO.write(new_record, 'Erdman.curated.gb', 'genbank')

1

# 5. Quality control

In [21]:
for feature in new_record.features:
    if feature.type == 'CDS':
        if 'pseudo' not in feature.qualifiers:
            aa = feature.qualifiers['translation']
            problem = ''
            
            if '*' in aa[:-1]:
                problem += 'Pseudo;'
            
            if aa[0] not in ['M', 'V', 'L']:
                problem += 'Start;'
            
            if aa[-1] != '*':
                problem += 'End;'
                
            if problem != '':
                print(problem)
                print(feature)

Start;
type: CDS
location: [550979:551153](-)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: gene, Value: ['mazE1']
    Key: inference, Value: alignment:ratt:1.0.3:RefSeq:NC_000962.3
    Key: locus_tag, Value: Erdman_0477
    Key: note, Value: Simliar to Rv0456B of M. tuberculosis H37Rv;ATT start inherited from H37Rv
    Key: product, Value: ['antitoxin MazE1']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ITTYYYVLLSVTTWVGLRHEAKRELVYRGRRSIGRMPREWACRRSRRFAANGVDAAR*

Start;
type: CDS
location: [837211:837739](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: gene, Value: ['ERv0742']
    Key: inference, Value: alignment:ratt:1.0.3:RefSeq:NC_000962.3
    Key: locus_tag, Value: Erdman_0779
    Key: note, Value: Simliar to Rv0742 of M. tuberculosis H37Rv;ATA start inherited from H37Rv
    Key: product, Value: ['hypothetical protein']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ISFVIAAPEAIAAAATDLASIGSTIGAANAAAAANTTAVLAAGADQVSVAIAAAFG

This are the three cases where the I start codon is inherited from H37Rv